# Fluxos de carbono | O sistema carbonato
**Interação Oceano-Atmosfera (11138)**
***
**Author:** [Andrés Piñango](https://github.com/andresawa/)  
Laboratório de Estudos dos Oceanos e Clima – LEOC, Instituto de Oceanografia, Universidade Federal do Rio Grande.  
email: andreseloy@furg.br  
**Last change:** 30/09/2021
***

## O fluxo de CO₂ na interface oceano-atmosfera
A troca de gases entre o oceano e a atmosfera é um processo importante no ciclo biogeoquímico de muitos elementos, como o CO₂ ou O₂. No caso do CO₂, a absorção de grandes quantidades deste gás proveniente das atividades antropogênicas pode alterar o equilíbrio do sistema carbonato do oceano, reduzindo a concentração do íon carbonato (e o OH⁻) e aumentando a concentração de prótons, em um processo conhecido como acidificação do oceano [(Doney et al., 2009)](https://doi.org/10.5670/oceanog.2009.93). Para compreender melhor essas alterações, é preciso conhecer quanto CO₂ os oceanos absorvem ou emitem num determinado período. Esse balanço entre emissão/absorção é chamado de fluxo líquido (FCO₂), e ele descreve o comportamento do oceano com relação à captação ou liberação de CO₂ atmosférico.  

Para estimar o fluxo de um gás através da interface ar-água, é necessário conhecer dois informações:
1) O desequilíbrio entre as concentrações do gás no ar e na superfície do oceano (controla a direção do fluxo)  
2) A taxa em que o desequilíbrio é removido (controla a intensidade do fluxo) 

Então, os fluxos de CO₂ em escalas regionais a globais são geralmente determinados usando a seguinte equação:  

$$
FCO_{2} = k \; K_{0} \; \left ( pCO_{2 \; oceano} - pCO_{2 \; ar} \right )
$$

Onde:
* $FCO_{2}$ é o fluxo de CO₂ (massa área⁻¹ tempo⁻¹)
* $k$ é o efeito da velocidade do vento na troca gasosa (comprimento tempo⁻¹)
* $K_{0}$ é a solubilidade do gás (massa volume⁻¹ pressão⁻¹)
* $pCO_{2}$ é a pressão parcial do CO₂ no mar e atmosfera

Em Python, $FCO_{2}$ pode ser calculado usando a seguinte função:

In [ ]:
def carbon_flux(gastransfer, solubility, pco2_sw, pco2_atm):
    result = gastransfer * solubility * (pco2_sw - pco2_atm)
    return result

Uma explicação mais detalhada dos parâmetros da equação do fluxo é apresentada abaixo.

### Efeito do vento: o coeficiente de transferência gasosa (*k*)
O incremento da velocidade do vento permite uma agitação mais vigorosa da superfície oceânica, aumentando a área de superfície, a formação de bolhas e outros processos, favorecendo a troca de gases. A relação entre a velocidade do vento e o fluxo de gás é empírica e existem várias formulações. Uma das mais utilizadas é a relação descrita por [Wanninkhof (2014)](https://doi.org/10.4319/lom.2014.12.351):  

$$
k = 0.251\; U^{2}\;  (Sc/660)^{-0.5}
$$

Onde: 
* $U$ é a velocidade do vento a uma altura de 10 metros acima da superfície oceânica.
* $Sc$ é o número de Schmidt, uma relação entre a taxa de difusão molecular de um gás em um fluido e a viscosidade desse fluido.
* $k$ é o efeito da velocidade do vento na troca gasosa (cm h⁻¹)  


> Fun fact: a parametrização de [Ho *et al.* (2006)](https://doi.org/10.1029/2006GL026817) é equivalente à parametrização de [Wanninkhof (2014)](https://doi.org/10.4319/lom.2014.12.351) e muito mais fácil de pronunciar 😆. Recomendo a leitura [desta](https://twitter.com/_david_ho_/status/1364846020778102788) sequência no Twitter.


O número de Schmidt (*Sc*) depende de cada gás, e pode ser calculado para água salgada (salinidade = 35) desde –2°C até 40°C usando a equação descrita em [Wanninkhof (2014)]( https://doi.org/10.4319/lom.2014.12.351): 

$$
Sc = A + Bt + Ct^{2} + Dt^{3} + Et^{4}
$$

Onde $t$ é a temperatura (°C), é os coeficientes para o CO₂ são:  

| Gás | A       | B       | C      | D         | E          |
|-----|---------|---------|--------|-----------|------------|
| CO₂ | 21116.8 | -136.25 | 4.7353 | -0.092307 | 0.00075555 |

Em Python, $k$ (em m ano⁻¹) pode ser calculado usando a seguinte função:

In [ ]:
def gastransfer_wanninkhof(temperature, wind_speed):
    A = 2116.8
    B = -136.25
    C = 4.7353
    D = -0.092307
    E = 0.0007555
    Sc = A + B * temperature + C * temperature**2 + D * temperature**3 + E * temperature**4
    convert = (24 * 365.25)/100 # Tranformar de (cm h⁻¹) a (m ano⁻¹)
    result = (0.251 * wind_speed**2 * (Sc / 660)**-0.5) * convert 
    return result

### O coeficiente de solubilidade do gás (*K₀*)

A solubilidade dum gás num liquido é proporcional à pressão parcial do mesmo, seguindo a [Lei de Henry](https://en.wikipedia.org/wiki/Henry%27s_law). As solubilidades dos gases dependem fortemente da temperatura e, em geral, eles são menos solúveis em temperaturas elevadas. A salinidade também tem um efeito na solubilidade dos gases, e geralmente salinidades elevadas encontra-se associadas a baixas solubilidades. A solubilidade do CO₂ e outros gases no oceano pode ser calculada mediante a equação descrita em [Weiss (1974)](https://doi.org/10.1016/0304-4203(74)90015-2): 

$$
K_{0} = e^{A1 +A2 \frac{100}{T} + A3\, ln \left ( \frac{T}{100} \right ) + S \left [B1 + B2 \frac{T}{100} + B3  (\frac{T}{100})^{2} \right ]}
$$

Onde:
* $T$ é a temperatura (Kelvin)
* $S$ é a salinidade 
* $K_{0}$ é a solubilidade (mol L⁻¹ atm⁻¹)  

é os coeficientes para o CO₂ são:  

| Gás | A1       | A2      | A3      | B1       | B2        | B3        |
|-----|----------|---------|---------|----------|-----------|-----------|
| CO₂ | -58.0931 | 90.5069 | 22.2940 | 0.027766 | -0.025888 | 0.0050578 |

Em Python, $K_{0}$ (em mol m⁻³ μatm⁻¹) pode ser calculado usando a seguinte função:

In [ ]:
def solubility_weiss(temperature, salinity):
    A1 = -58.0931
    A2 = 90.5069
    A3 = 22.2940
    B1 = 0.027766
    B2 = -0.025888
    B3 = 0.0050578
    tempK = temperature + 273.15 # °C a K
    solubility =  np.exp(
        A1 + 
        A2 * (100/tempK) + 
        A3 * np.log(tempK/100) + 
        salinity * (B1 + 
                    B2 * (tempK/100) + 
                    B3 * (tempK/100)**2
                   )
    )
    convert = 1/1000 # # Tranformar de (mol L⁻¹ atm⁻¹) a (mol m⁻³ μatm⁻¹)
    result = solubility * convert
    return result

***
## Atividade 1. Calculo do fluxo de CO₂ em duas regiões do Oceano Atlântico

**Objetivo:** Calcular o fluxo líquido de CO₂ entre o oceano e a atmosfera (FCO₂) em duas regiões do Oceano Atlântico (região subtropical norte e região tropical) desde o ano 2000 até o 2020 e identificar se essas regiões são uma fonte ou um sumidouro de CO₂ atmosférico.  

**Procedimento:** Os alunos devem utilizar as equações apresentadas acima para estimar o fluxo de CO₂ (em $\frac{mmol}{m^{2}\, dia}$) nos seguintes pontos:

|  Ponto  | Temperatura  (°C) | Salinidade | pCO₂ oceano (μatm) | Velocidade Vento (m/s) | pCO₂ atmosfera (μatm) |
|:-------:|:-----------------:|:----------:|:------------------:|:----------------------:|:---------------------:|
| Ponto 1 |        2.45       |    34.97   |       341.55       |          8.22          |         413.25        |
| Ponto 2 |       14.78       |    35.15   |       414.96       |          7.31          |         401.08        |
| Ponto 3 |       17.27       |    36.65   |       280.13       |         10.76          |         333.78        |
| Ponto 4 |       25.74       |    35.35   |       326.96       |          6.84          |         330.08        |
| Ponto 5 |       -2.37       |    30.44   |       400.13       |          9.74          |         390.76        |

Os resultados devem ser inseridos na seguinte bloco de código, que irá validar se a resposta é correta. Se o cálculo estiver correto, os alunos poderão usar o código em anexo que calculará o fluxo de CO₂ durante os últimos 20 anos automaticamente para as duas regiões avaliadas no oceano Atlântico. Uma descrição detalhada de como os dados foram preparados, incluindo as fontes utilizadas, pode ser encontrada [aqui](https://mybinder.org/v2/gh/andresawa/fluxes_exercice/main?filepath=data_preparation.ipynb).

**Resultados Esperados**: Neste trabalho, espera-se que o aluno entenda como o fluxo de CO₂ entre a interface ar-mar é calculado a partir dos dados fornecidos. Além disso, é esperado que o aluno explique os resultados encontrados de forma crítica, considerando todo o conhecimento adquirido nas últimas aulas de Interação Oceano Atmosfera. Isso será demonstrado através do relatório que o aluno fará, respondendo a perguntas presentes no bloco **questões**, situado no final deste documento.

#### Inserir os fluxo de CO₂ calculados por você aqui (só usar 3 decimais):

In [ ]:
Fco2_ponto1 = 0.000
Fco2_ponto2 = 0.000
Fco2_ponto3 = 0.000
Fco2_ponto4 = 0.000
Fco2_ponto5 = 0.000
exec(open("test.py").read())

#### Se o resultado é correto, você pode usar este código para calcular os fluxos nas duas regiões desde o 2000 até o 2020

In [ ]:
# Importa as librerias
import numpy as np
import pandas as pd
import xarray as xr

# Importa os dados das duas regiões
if (teste):
    data_north = xr.open_dataset("north_data.nc")
    data_equator = xr.open_dataset("equator_data.nc")

# Calcula o efeito da velocidade do vento no fluxo de CO₂ (k)
k_north = xr.apply_ufunc(gastransfer_wanninkhof, data_north.temperature, data_north.wind_speed)
k_equator = xr.apply_ufunc(gastransfer_wanninkhof, data_equator.temperature, data_equator.wind_speed)

# Calcula o coeficiente de solubilidade (K₀)
K0_north = xr.apply_ufunc(solubility_weiss, data_north.temperature, data_north.salinity)
K0_equator = xr.apply_ufunc(solubility_weiss, data_equator.temperature, data_equator.salinity)

# Calcula os fluxos de CO₂ nas duas regiões (FCO₂)
fluxo_north = xr.apply_ufunc(carbon_flux, k_north, K0_north, data_north.fco2, data_north.pco2_atm)
fluxo_equator = xr.apply_ufunc(carbon_flux, k_equator, K0_equator, data_equator.fco2, data_equator.pco2_atm)

# Calcula a média de FCO₂ na região para todos os meses
Fco2_north = fluxo_north.mean(["latitude", "longitude"])
Fco2_equator = fluxo_equator.mean(["latitude", "longitude"])

# Calcular a média para todo o período estudado
mean_north = Fco2_north.mean().data
mean_equator = Fco2_equator.mean().data

***
## Atividade 2. O sistema carbonato: calculo do pH a partir de pCO₂ e alcalinidade

**Objetivo:** Calcular o pH das águas superficiais a partir dos valores de pCO₂ e alcalinidade nas duas regiões do Oceano Atlântico estudadas, desde o ano 2000 até o 2020, e identificar se existem variações neste parâmetro.  

**Procedimento:** Os parâmetros do sistema carbonato podem ser calculados a partir de dois variáveis (pCO₂, pH, DIC ou TA) e dados de temperatura, salinidade e pressão (ou profundidade), utilizando diversos pacotes. Neste exercício, os alunos devem calcular a alcalinidade potencial das aguas superficiais (alcalinidade potencial = alcalinidade total + concentração nitrato) mediante as relações calculadas por [Takahashi et al. (2014)](https://doi.org/10.1016/j.marchem.2014.06.004), e depois, a partir da alcalinidade total calculada e os dados de fCO₂ disponíveis, calcular o pH mediante o pacote [PyCO2SYS](10.5281/zenodo.5176106). O código para fazer todos os cálculos encontra-se no seguinte bloco. As relações de Takahashi para as regiões estudadas, assim como a média de nitrato proveniente do [World Ocean Atlas 2018](https://www.ncei.noaa.gov/products/world-ocean-atlas) podem ser encontrados na seguinte tabela:

|               Região              | Relação Alcalinidade Potencial - Salinidade | Concentração Nitrato (μmol/kg) |
|:---------------------------------:|:-------------------------------------------:|:------------------------------:|
|    Atlântico Norte (40°N-60°N)    |     PALK = (45.30 x Salinidade) + 733.0     |               4.4              |
| Altlântico Equatorial (10°S-10°N) |     PALK = (58.25 x Salinidade) + 270.9     |               0.3              |

**Resultados Esperados**: Neste trabalho, espera-se que o aluno entenda como os parâmetros do sistema carbonato (exemplo: o pH) é calculado a partir dos dados fornecidos. Além disso, é esperado que o aluno explique os resultados encontrados de forma crítica, considerando todo o conhecimento adquirido nas últimas aulas de Interação Oceano Atmosfera. Isso será demonstrado através do relatório que o aluno fará, respondendo a perguntas presentes no bloco **questões** situado no final deste documento.

In [ ]:
# Importa as librerias
import PyCO2SYS as pyco2

# Função para calcular a alcalinidade (Alk) a partir da relação alcalinidate potencial (PAlk) - Salinidade e a concentração de nitrato
def alkalinity(salinity, region):
    if (region == "north_atlantic"):
        palk = (45.30 * salinity) + 733.0
        result = palk - 4.4
        return result
    elif (region == "equatorial_atlantic"):
        palk = (58.25 * salinity) + 270.9
        result = palk - 0.3
        return result
    else:
        print('A região introducida não é correta, os possíveis valores são "north_atlantic" ou "equatorial_atlantic". Por favor tente novamente.')

# Calcula a alcalinidade (TA) para as duas regiões
alk_north = xr.apply_ufunc(alkalinity, data_north.salinity, "north_atlantic")
alk_equator = xr.apply_ufunc(alkalinity, data_equator.salinity, "equatorial_atlantic")

# Função para calcular o pH
def pH_calc(alkalinity_sw, fco2_sw, temperature_sw, salinity_sw):
    carbonate_system = pyco2.sys(par1 = alkalinity_sw, 
                                 par2 = fco2_sw, 
                                 par1_type = 1, 
                                 par2_type = 5, 
                                 salinity = salinity_sw,
                                 temperature = temperature_sw,
                                 opt_gas_constant = 1
                                )
    result = carbonate_system["pH_total"]
    return result

# Calcula o pH
pH_north = xr.apply_ufunc(pH_calc, alk_north, data_north.fco2, data_north.temperature, data_north.salinity)
pH_equator = xr.apply_ufunc(pH_calc, alk_equator, data_equator.fco2, data_equator.temperature, data_equator.salinity)

# Calcula a média para as duas regiões
mpH_north = pH_north.mean(["latitude", "longitude"])
mpH_equator = pH_equator.mean(["latitude", "longitude"])

# Calcula o valor medio de FCO₂ na região agrupado por mes
monthly_pH_north = pH_north.groupby("time.month").mean(["latitude", "longitude", "time"])
monthly_pH_equator = pH_equator.groupby("time.month").mean(["latitude", "longitude", "time"])

# Calcula a anomalia de FCO₂ para todos os meses
apH_north = mpH_north.groupby("time.month") - monthly_pH_north
apH_equator = mpH_equator.groupby("time.month") - monthly_pH_equator

# Calcula a taxa (trend) da acidificação com as anomalias
idx_north = np.isfinite(mpH_north)
trend_north = np.polyfit(np.arange(0, apH_north.shape[0])[idx_north], apH_north.values[idx_north], 1)
idx_equator = np.isfinite(mpH_equator)
trend_equator = np.polyfit(np.arange(0, apH_equator.shape[0])[idx_equator], apH_equator.values[idx_equator], 1)

***
## Resultados
Execute este código para gerar uma figura com os principais resultados das atividades 1 e 2. Utilize esta figura para responder as questões presentes no final deste documento.

In [ ]:
# Importar a libreria para as figuras
import matplotlib.pyplot as plt

# Graficar a serie temporal dos fluxos de CO₂
fig, ax = plt.subplots(3, 1, figsize=(10, 8), dpi = 300)

# Show the first subplot
ax[0].plot(Fco2_north.time, Fco2_north.values, c='g')
ax[0].axhline(mean_north, c = "g", alpha = 0.3, linestyle = '--', label = f"Média: {mean_north.round(decimals=2)}")
ax[0].plot(Fco2_equator.time, Fco2_equator.values, c='r')
ax[0].axhline(mean_equator, c = "r", alpha = 0.3, linestyle = '--', label = f"Média: {mean_equator.round(decimals=2)}")
ax[0].axhline(0, c = "black", alpha = 0.7)
ax[0].set_ylabel("FCO₂ [mol C m⁻² y⁻¹]")
ax[0].legend()

# Show the second subplot
ax[1].plot(data_north.time, data_north.fco2.mean(["latitude", "longitude"]), c='g', label = "Atlântico Norte")
ax[1].plot(data_equator.time, data_equator.fco2.mean(["latitude", "longitude"]), c='r',  label = "Atlântico Tropical")
ax[1].plot(data_equator.sel(time = slice("2001", "2020")).time, 
           data_equator.sel(time = slice("2001", "2020")).pco2_atm, c='black', alpha = 0.7,  label = "Atmosfera")
ax[1].set_ylabel("pCO₂ [μatm]");
ax[1].legend()

# Show the third subplot
ax[2].plot(mpH_north.time, mpH_north.values, c='g')
ax[2].plot(mpH_equator.time, mpH_equator.values, c='r')
ax[2].plot(mpH_north.time[12:], 
           (np.arange(12, apH_north.shape[0]) * trend_north[0]) + 8.11, 
           c='g', alpha = 0.3, linestyle = '--', label = f"Taxa: {round(trend_north[0]*120, 3)} decade⁻¹")
ax[2].plot(mpH_equator.time[12:], 
           (np.arange(12, apH_equator.shape[0]) * trend_equator[0]) + 8.07, 
           c='r', alpha = 0.3, linestyle = '--', label = f"Taxa: {round(trend_equator[0]*120, 3)} decade⁻¹")
ax[2].set_ylabel("pH [total scale]")
ax[2].set_xlabel("Time [years]")
ax[2].legend();

***
## Questões
* **Questão 1 (Exame):** Baseado nos resultados obtidos para o FCO₂, qual o comportamento das regiões oceânicas avaliadas (áreas emissoras ou absorvedoras de CO₂)? Comente sobre as causas principais que causam os comportamentos encontrados nas duas regiões.

* **Questão 2 (Exame):** Os resultados obtidos para o FCO₂ das duas regiões avaliadas estão de acordo com a [literatura/trabalhos pretéritos](https://pure.mpg.de/rest/items/item_3184344/component/file_3205962/content)? Comente.

* **Questão 3 (Exame):** Indique a causa e explique o processo de acidificação dos oceanos. Baseado no resultado obtido para o pH da água do mar, as regiões avaliadas estão sofrendo com o processo de acidificação? Comente.

* **Questão 4 (Exame):** As variações temporais do pH da água do mar são similares nas duas regiões avaliadas? Se existir diferenças, o que pode estar causando isso?

* **Questão 5 (Exame):** Os resultados obtidos para o pH das águas superficiais das duas regiões avaliadas estão de acordo com a [literatura/trabalhos préteritos](https://academiccommons.columbia.edu/doi/10.7916/D8QF94M5/download)? Comente.